# Rechunking 

In [3]:
PATH = "/s3/scratch/wessim.omezzine/"
PATH = "./"
import xarray as xr

In [11]:
import xarray as xr
from rechunker import rechunk

ds = xr.open_zarr("s3://wfp-ops-userdata/public-share/ndvi_world.zarr")
ds  = ds.isel(time=slice(0,10), latitude=slice(0,None), longitude=slice(0,None))


# Define the new chunk sizes
new_time_chunk_size = -1
new_latitude_chunk_size = 50
new_longitude_chunk_size = 50
target_chunks = {'time':
                 new_time_chunk_size, 'latitude': new_latitude_chunk_size, 'longitude': new_longitude_chunk_size}




intermediate_store = PATH + 'ndvi_intermediate_store.zarr'  # Update with your desired path

target_store = PATH + 'ndvi_target_store.zarr'  # Update with your desired path


max_mem = "12GB"


In [4]:
! rm -r ndvi_intermediate_store.zarr ndvi_target_store.zarr/

In [5]:
# Perform rechunking
rechunk_plan = rechunk(ds, target_chunks, max_mem, target_store, temp_store=intermediate_store)
result = rechunk_plan.execute()

In [ ]:
import xarray as xr

ds_full = xr.open_zarr("s3://wfp-ops-userdata/public-share/ndvi_world.zarr")


# Define the new chunk sizes
new_time_chunk_size = -1
new_latitude_chunk_size = 50
new_longitude_chunk_size = 50
target_chunks = {'time':
                 new_time_chunk_size, 'latitude': new_latitude_chunk_size, 'longitude': new_longitude_chunk_size}

max_mem = "12GB"

time_step_size = 30  # Define batch size
num_batches = ds_full.dims['time'] // time_step_size


concatenated_ds_list = []

for i in range(num_batches):
    target_store = PATH + f'NDVI Rechunked/ndvi_target_store_batch_{i}.zarr'
    ds_rechunked = xr.open_zarr(target_store)
    concatenated_ds_list.append(ds_rechunked)

# Concatenate all datasets along the time dimension
combined_ds = xr.concat(concatenated_ds_list, dim='time')


# Combine rechunked datasets

In [ ]:
PATH = "./"

ds_full = xr.open_zarr("s3://wfp-ops-userdata/public-share/ndvi_world.zarr")

new_time_chunk_size = -1
new_latitude_chunk_size = 50
new_longitude_chunk_size = 50
target_chunks = {'time':
                 new_time_chunk_size, 'latitude': new_latitude_chunk_size, 'longitude': new_longitude_chunk_size}

max_mem = "12GB"

time_step_size = 30  # Define batch size
num_batches = ds_full.dims['time'] // time_step_size


concatenated_ds_list = []

for i in range(num_batches):
    target_store = PATH + f'NDVI Rechunked/ndvi_target_store_batch_{i}.zarr'
    ds_rechunked = xr.open_zarr(target_store)
    concatenated_ds_list.append(ds_rechunked)

NDVI = xr.concat(concatenated_ds_list, dim='time')


concatenated_ds_list = []

for i in range(num_batches):
    target_store = PATH + f'RFH Rechunked/rfh_target_store_batch_{i}.zarr'
    ds_rechunked = xr.open_zarr(target_store)
    concatenated_ds_list.append(ds_rechunked)

# Concatenate all datasets along the time dimension
RFH = xr.concat(concatenated_ds_list, dim='time')


# LTA

In [ ]:
NDVI_lta = NDVI.sel(time=slice("2003-02-01","2018-12-31"))
NDVI_lta  = NDVI_lta .groupby(NDVI_lta.time.dt.strftime("%m-%d")).mean()
NDVI_lta = NDVI_lta.sel(strftime=NDVI.time.dt.strftime("%m-%d"))

RFH_lta = RFH.sel(time=slice("2003-02-01","2018-12-31"))
RFH_lta  = RFH_lta .groupby(RFH_lta.time.dt.strftime("%m-%d")).mean()
RFH_lta = RFH_lta.sel(strftime=RFH.time.dt.strftime("%m-%d"))

In [10]:
NDVI_lta

<xarray.Dataset>
Dimensions:      (time: 720, latitude: 3600, longitude: 7200)
Coordinates:
  * latitude     (latitude) float64 89.97 89.92 89.88 ... -89.88 -89.93 -89.98
  * longitude    (longitude) float64 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
    spatial_ref  int32 0
    strftime     (time) object '07-01' '07-11' '07-21' ... '06-11' '06-21'
  * time         (time) datetime64[ns] 2002-07-01 2002-07-11 ... 2022-06-21
Data variables:
    band         (time, latitude, longitude) float64 dask.array<chunksize=(1, 50, 50), meta=np.ndarray>